# Compare the pipeline with and without rigid pre-alignment

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import pandas as pd
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.feature_detection.utils_features import *
from DLC_for_WBFM.utils.feature_detection.utils_tracklets import *
from DLC_for_WBFM.utils.feature_detection.utils_detection import *
from DLC_for_WBFM.utils.feature_detection.visualization_tracks import *
from DLC_for_WBFM.utils.feature_detection.feature_pipeline import *
from DLC_for_WBFM.utils.feature_detection.utils_affine import *
from DLC_for_WBFM.utils.feature_detection.class_reference_frame import PreprocessingSettings


In [2]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)

In [3]:
# Same opt for both
opt = {'start_frame':0,
       'num_frames':2,
       'verbose':2}

old_matches, old_conf, old_frames = track_neurons_full_video(btf_fname_red, **opt)

preprocessing_settings = PreprocessingSettings()
preprocessing_settings.do_filtering=False
preprocessing_settings.do_rigid_alignment=True
new_matches, new_conf, new_frames = track_neurons_full_video(btf_fname_red, **opt, preprocessing_settings=preprocessing_settings)

Building initial frame...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.41s/it]


Building initial frame...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:16<00:00, 16.09s/it]


In [4]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


## TODO: compare neuron tubes

## Visualize the neurons found

In [5]:
which_frame = 1

f_old = old_frames[which_frame]
n_old, pc_old, _ = build_neuron_tree(f_old.neuron_locs)
pc_old.paint_uniform_color([0.5,0.5,0.5])

f_new = new_frames[which_frame]
n_new, pc_new, _ = build_neuron_tree(f_new.neuron_locs)
pc_new.paint_uniform_color([0,1,0])

PointCloud with 137 points.

In [6]:
o3d.visualization.draw_geometries([pc_old, pc_new])

## Visualize the matches

In [ ]:
which_pair = (0,1)

old_lines = plot_matched_point_clouds(old_frames,old_matches,which_pair, [0,0,1], False)

In [ ]:
new_lines = plot_matched_point_clouds(new_frames,new_matches,which_pair,[0,1,0], False)

In [ ]:
o3d.visualization.draw_geometries(old_lines + new_lines)